In [2]:
from IPython.core.debugger import set_trace
import json
from collections import Counter
import pandas as pd
from tqdm.autonotebook import tqdm
tqdm.pandas()

from django.db import connection
import os
import sys
import re
import time
import string
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

import django
django.setup()

from data.models import Newsdata
import glob

import spacy
nlp = spacy.load('en_core_web_lg')

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\tqdm\_tqdm.py:604: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def preproc(text, remove_url=True, remove_mention=True, remove_hashtag=False):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = '((?: rt)|(?:^rt))[^ @]?'
    EMOJI = r'[\U00010000-\U0010ffff]'
    DOTS = '…'
    LONG_BLANK = r'[ ]+'
    SPECIALS = r'([^ a-zA-Z0-9_\u3131-\u3163\uac00-\ud7a3]+)|([ㄱ-ㅣ]+)'
    PUNC = r'[{}]'.format(string.punctuation)
    
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
    URL = '|'.join((URL1, URL2))
    
    HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
    MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
    #PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    #out = {}
    #text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS)), '', text.lower())
    text = text.lower()
    
    if remove_url:
        text = re.sub(URL, ' ', text)

    if remove_mention:
        text = re.sub(MENTION, ' ', text)        
    else:
        text = re.sub(MENTION, ' \g<inner_mention>', text)
        
    if remove_hashtag:
        text = re.sub(HASHTAG, ' ', text)
    else:
        text = re.sub(HASHTAG, ' \g<inner_hashtag>', text)
        
    text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS, PUNC)), ' ', text)
    return re.sub(LONG_BLANK, ' ', text).strip()

In [3]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS; len(stopwords)

326

In [4]:
fnames = glob.glob('newsdata/downloaded/*.json'); len(fnames)

13652

In [5]:
Newsdata.objects.all().delete()

(0, {'data.Newsdata': 0})

In [6]:
for i, fname in enumerate(tqdm(fnames)):
    with open(fname, encoding='UTF-8-sig') as f:
        content = json.load(f)
        doc = nlp(preproc(content['text']))
        
        obj = Newsdata()
        obj.id = i + 1
        obj.pub = content['pub']
        obj.authors = content['authors']
        obj.title = content['title']
        obj.published_at = pd.Timestamp(content['published_at'], tz='utc')
        obj.downloaded_at = pd.Timestamp(content['downloaded_at'], tz='utc')
        obj.text = content['text']
        obj.description = content['description']
        obj.top_image = content['top_image']
        obj.url = content['url']
        obj.words = [str(noun) for noun in doc.noun_chunks if str(noun) not in stopwords]
        obj.language = content['language']
        obj.save()

In [33]:
objs = Newsdata.objects.all().iterator()
n = objs.count()
n_done = [0]
status = '\r{pct}% completed'

def progress():
    n_done[0] += 1
    pct = '%.2f' % (n_done[0] / n * 100)
    print(status.format(pct=pct), end='')

for obj in objs:
    obj.authors = obj.authors.split(',')
    obj.save()
    progress()
    #print(obj.authors)

10.00% completed['']
20.00% completed['The Associated Press']
30.00% completed['Mary Novakovich']
40.00% completed['The Washington Times Http', 'Nasser Karimi', 'Jon Gambrell']
50.00% completed['Andy Bruce', 'Min Read']
60.00% completed['Lucy Daltroff']
70.00% completed['Dante Carrer', 'Min Read']
80.00% completed['Ben Chu']
90.00% completed['Reporter']
100.00% completed['']


In [5]:
from collections import Counter
words = Newsdata.objects.all()[:5].values_list('words', flat=True)
counter = Counter()
[counter.update(item) for item in words]

[None, None, None, None, None]

In [6]:
counter.most_common(100)

[('xero', 9),
 ('nz', 6),
 ('this multiple choice challenge shares', 5),
 ('the video', 5),
 ('small businesses', 4),
 ('a video', 4),
 ('his earwax', 4),
 ('the month', 3),
 ('the year', 3),
 ('the company', 3),
 ('people', 3),
 ('twitter', 3),
 ('the rapper', 3),
 ('the indices', 2),
 ('the end', 2),
 ('china trade', 2),
 ('the trading range', 2),
 ('traders', 2),
 ('september', 2),
 ('the potential', 2),
 ('javale mcgee', 2),
 ('burglars', 2),
 ('tmz', 2),
 ('mcgee', 2),
 ('the men', 2),
 ('advisors', 2),
 ('data', 2),
 ('the 2019 financial year', 2),
 ('this multiple choice challenge', 2),
 ('shares', 2),
 ('a fan', 2),
 ('his own earwax', 2),
 ('a lot', 2),
 ('social media', 2),
 ('west', 2),
 ('his finger', 2),
 ('his ear', 2),
 ('a couple days', 1),
 ('positive trading', 1),
 ('the month optimistic comments', 1),
 ('end', 1),
 ('the month window dressing', 1),
 ('allocations', 1),
 ('the top', 1),
 ('early august', 1),
 ('the 50 day simple moving average', 1),
 ('the market', 1)

In [84]:
from tqdm.autonotebook import tqdm
tqdm.pandas()
import json
import glob
import os
import hashlib

fnames = glob.glob('newsdata/downloaded/*.json')
ext = '.json'

_fnames = []

for fname in tqdm(fnames):
    
    with open(fname, encoding='UTF-8-sig') as f:
        content = json.load(f)
        pub = content['pub']
        url = content['url']
        pubdate = pd.Timestamp(content['published_at'], tz='utc')
        downdate = pd.Timestamp(content['downloaded_at'], tz='utc')

        if pubdate.date() < downdate.date():
            print(url)
        
        
#         if (pub=='reuters' and '?il=0' in url) or (pub=='marketwatch' and '?mod=' in url):
#             _url = clean_url(pub, url)
#             hash_url = hashlib.sha1(_url.encode('utf-8')).hexdigest()
#             file = os.path.join('newsdata/downloaded', hash_url + ext)
#             if file in fnames:
#                 fnames_to_remove.append(fname)

https://realmoneypro.thestreet.com/investing/beware-the-ides-of-september-15073256
https://sports.yahoo.com/report-ja-vale-mc-gee-robbed-of-championship-ring-200-k-in-cash-jewelry-in-home-burglary-215250390.html
https://washingtontimes.com/quiz/2018/dec/5/you-name-these-1950s-songs-lyrics
https://businessinsider.co.za/kanye-west-earwax-sunday-service-2019-9
https://washingtontimes.com/quiz/2015/jun/16/i-declare-2016
https://townhall.com/columnists/johnandandyschlafly/2019/08/28/finally-a-president-stands-up-to-china-n2552287
https://independent.co.uk/news/world/middle-east/gulf-of-oman-attacks-iran-oil-who-response-latest-a8962621.html
https://politico.eu/article/johnson-brexit-brexit-plan-has-a-hole-in-it-so-big-you-can-swim-through-it
https://bbc.co.uk/news/world-europe-49541111
https://hotair.com/archives/taylormillard/2019/08/31/china-booted-wsj-reporter-following-story-xis-cousin
https://independent.co.uk/sport/football/premier-league/liverpool-vs-burnley-sadio-mane-mohamed-salah-

https://washingtonpost.com/weather/2019/09/01/catastrophic-hurricane-dorian-unleashes-devastating-blow-northern-bahamas-takes-aim-southeast-us
https://npr.org/sections/health-shots/2019/07/14/741310765/simple-ways-to-prevent-falls-in-older-adults
https://www.zdnet.com/article/whos-best-at-updating-android-phones-nokia-leads-for-pie-upgrades-security-fixes
https://businessinsider.com/islamic-state-using-cow-suicide-bomb-ied-2019-9
https://cbsnews.com/news/bernie-sanders-2020-campaign-manager-faiz-shakir-its-absolutely-critical-we-win-new-hampshire
https://wsj.com/articles/brexit-battle-lines-harden-as-lawmakers-prepare-for-parliamentary-showdown-11567432609?mod=rsswn
https://marketwatch.com/story/asian-markets-mixed-as-latest-us-china-tariffs-take-effect-2019-09-01
https://investing.com/news/economy-news/comic-markets-freak-out-as-big-fed-rate-cut-bets-fade-1918322
https://townhall.com/watchdog/wisconsin/2017/09/15/wisconsin-focus-senate-plans-take-state-budget-without-votes-pass-n10947

https://independent.co.uk/extras/lifestyle/pizza-dough-cancer-charity-mount-snowdon-a9055011.html
https://www.zdnet.com/article/karthik-rau-ceo-of-signalfx-on-new-approaches-to-monitoring-cloud-application-microservices
https://washingtontimes.com/news/2019/sep/2/alexandria-ocasio-cortez-ayanna-pressley-push-fund
https://npr.org/2019/03/07/701212707/logic-logic-chris-zarou
https://cnn.com/2019/07/22/health/plant-based-diet-diabetes-study/index.html
https://news.yahoo.com/check-reservation-american-airlines-extends-141430360.html
https://news.yahoo.com/iran-tanker-centre-diplomatic-row-114801451.html
https://foxbusiness.com/industrials/tesla-model-s-crashes-catches-fire-in-russia
https://cbsnews.com/news/democratic-debates-here-are-the-candidates-who-have-qualified-for-the-2020-democratic-debates
https://nymag.com/press/2017/08/vulture-and-the-cut-launching-shows-on-facebooks-watch.html
https://washingtontimes.com/news/2019/sep/2/letters-to-the-editor-rid-government-of-deep-state
https:

https://bleacherreport.com/articles/2852338-ezekiel-elliott-deal-may-be-only-the-start-of-headaches-for-cowboys-jerry-jones
https://cnn.com/2019/07/29/business/daimler-electric-trucks/index.html
https://washingtontimes.com/quiz/2018/may/17/how-much-do-you-know-about-cia
https://independent.co.uk/news/business/news/us-china-trade-war-trump-new-tariffs-xi-jinping-expensive-a9087391.html
https://npr.org/2019/01/26/688998177/fatal-end-to-an-alleged-game-of-russian-roulette-for-st-louis-police-officer
https://uk.reuters.com/article/uk-nissan-ceo/nissan-ceo-saikawa-admits-to-misconduct-in-compensation-jiji-idUKKCN1VP34D
https://independent.co.uk/money/spend-save/what-expect-sajid-javid-spending-review-chancellor-tax-brexit-snap-general-election-a9084626.html
https://radio.foxnews.com/2019/08/29/the-legacy-of-jim-and-elisabeth-elliot-a-love-story-for-the-ages
https://thinkprogress.org/britains-pm-boris-johnson-suspends-parliament-so-he-can-force-through-his-brexit-vision-82b8d56752a9/
https:/

https://abcnews.go.com/GMA/Food/easy-upgrades-hamburgers-hot-dogs-level-labor-day/story?id=65261667
https://washingtonpost.com/opinions/caucuses-are-outdated-and-irrational-replace-them-with-primaries/2019/09/02/d37a6264-cb52-11e9-be05-f76ac4ec618c_story.html
https://finance.townhall.com/columnists/mikeshedlock/2019/08/21/another-trump-trade-fiasco-this-time-with-japan-n2552013
https://radio.foxnews.com/2019/09/04/the-todd-starnes-show-september-4th-2019
https://townhall.com/columnists/derekhunter/2019/08/29/its-time-cnn-drops-an-n-n2552314
https://businessinsider.my/trump-trade-war-us-and-china-escalate-with-more-tariffs-2019-8
https://investing.com/news/technology-news/israels-cortica-teams-with-toyota-bmw-in-autonomous-ai-car-tech-1969533
https://independent.co.uk/extras/indybest/outdoor-activity/best-active-swimsuit-bikini-tankini-support-one-piece-two-surfing-paddleboarding-swimming-a9021276.html
https://cnn.com/2019/09/02/asia/china-school-hubei-dead-intl-hnk/index.html
https://i

...

https://politico.eu/article/johnson-does-want-an-election
https://newrepublic.com/article/154890/scourge-worker-wellness-programs
https://cnn.com/2019/09/04/business/us-china-trade-war-talks/index.html
https://npr.org/2019/08/07/748972017/news-brief-el-paso-victims-boy-scouts-sex-abuse-allegations-hong-kong
https://townhall.com/watchdog/Iowa/2016/08/23/losing-supreme-court-iowa-felons-turn-legislature-search-voting-rights-n9793
https://businesstimes.com.sg/garage/grab-and-gojek-squaring-off-in-international-food-fight 
https://independent.co.uk/money/spend-save/too-expensive-claim-insurance-car-insurance-premiums-excess-policy-a9075506.html
https://heritage.org/asia/commentary/us-needs-respond-rising-chinese-influence-the-united-nations
https://community.morningstar.com/t5/TIAA-Funds/Plan-Changes/td-p/20112/jump-to/first-unread-message
https://wsj.com/articles/chinas-red-hot-tobacco-stock-is-cooling-11567346469?mod=rsswn
https://thecut.com/2019/08/paul-hollywood-instagram-great-british

https://cnn.com/2019/08/09/health/philadelphia-eagles-stadium-sensory-room/index.html
https://thinkprogress.org/environmental-racism-is-bad-for-your-brain-b39753c3c61c/
https://foxnews.com/food-drink/tennessee-man-sues-popeyes-over-fruitless-hunt-for-sold-out-chicken-sandwich
https://sports.yahoo.com/wainwright-leads-surging-cardinals-3-1-win-over-211819798--mlb.html
https://townhall.com/watchdog/Iowa/2017/01/04/school-choice-prospects-looking-iowa-n10242
https://cnn.com/2019/07/20/health/iyw-cities-losing-36-million-trees-how-to-help-trnd/index.html
https://bbc.co.uk/news/business-49394848
https://bbc.com/news/stories-49557993
https://thinkprogress.org/taylor-swift-supports-the-equality-act-which-the-lgbtq-community-desperately-needs-right-now-b6c175bab475/
https://nymag.com/strategist/article/best-tube-socks.html
https://foxbusiness.com/healthcare/hurricane-dorian-power-outages-elderlys-threat-dr-marc-siegel
https://independent.co.uk/extras/indybest/us/the-nosmudge-lipstick-that-cost

https://cbsnews.com/news/seth-moulton-interview-2020-democratic-hopeful-says-barr-should-resign-because-he-disobeyed-the-law
https://townhall.com/health/sleep/2018/04/19/valerian-gabas-benefits-sleep-n11781
https://finance.townhall.com/columnists/mikeshedlock/2019/08/27/trump-claims-meaningful-talks-at-the-highest-level-china-doesnt-confirm-n2552248
https://news.yahoo.com/brother-british-woman-killed-expat-172654524.html
https://washingtontimes.com/news/2019/sep/4/redskins-downplay-significance-adding-former-eagle
https://npr.org/2019/07/06/739224759/jo-o-gilberto-master-of-bossa-nova-dies-at-88
https://independent.co.uk/voices/brexit-boris-johnson-jeremy-corbyn-labour-party-momentum-referendum-final-say-a8861516.html
https://businessinsider.com/bricks-of-cocaine-appearing-on-florida-beaches-from-hurricane-dorian-2019-9
https://news.yahoo.com/u-special-envoy-shares-peace-110442340.html
https://politico.com/story/2019/09/04/andrew-miller-roger-stone-mueller-1481320
https://nationalrevie

https://npr.org/sections/health-shots/2019/05/07/721005929/no-visible-bruises-upends-stereotypes-of-abuse-sheds-light-on-domestic-violence
https://in.reuters.com/article/usa-afghanistan-concerns/in-trumps-team-misgivings-emerge-over-any-deal-with-taliban-in-afghanistan-u-s-officials-idINKCN1VK2O0
https://independent.co.uk/environment/climate-crisis-net-zero-eat-meat-buy-clothes-carbon-footprint-travel-a9083681.html
https://cnbc.com/2019/09/04/no-deal-brexit-could-be-the-end-of-the-beginning-for-the-uk.html
https://cbsnews.com/news/greta-thunberg-new-york-climate-change-activist-reaches-nyc-after-sailing-across-atlantic-today-2019-08-28
https://politico.eu/event/politicos-eu-studies-fair-20th-edition
https://sports.yahoo.com/mlb-gpp-pivots-sunday-9-142743822.html
https://townhall.com/health/blood-sugar/2018/03/26/diabetogens-toxins-that-cause-diabetes-n11775
https://heritage.org/immigration/commentary/when-illegal-aliens-break-the-law
https://cnn.com/2019/08/16/us/gallery/el-paso-victim

https://thehill.com/opinion/white-house/459006-how-trump-can-avoid-a-gop-fumble-on-health-care-that-could-derail-his
https://cnn.com/2019/08/29/opinions/public-service-stacey-abrams-opinion-psaki/index.html
https://washingtontimes.com/news/2019/aug/29/dc-area-schools-open-new-leadership-facilities
https://townhall.com/health/arthritis/2018/05/24/top-5-supplements-fight-arthritis-n11793
https://npr.org/2019/08/31/754429787/mind-games-delivers-fast-paced-tech-thrills
https://washingtonpost.com/science/2019/09/02/humans-shaped-dogs-bodies-we-also-altered-their-brains
https://uk.reuters.com/article/uk-britain-eu/johnson-threatens-election-ahead-of-brexit-battle-in-parliament-idUKKCN1VN0BS
https://washingtonpost.com/national/in-trump-country-a-group-of-coal-miners-rebel-over-lost-jobs-missed-paychecks/2019/08/31/52ee4fbc-cb2e-11e9-a4f3-c081a126de70_story.html
https://independent.co.uk/travel/africa/elephant-pachyderms-tsavo-maasai-kenya-poaching-death-toll-a9030251.html
https://washingtonti

In [47]:
from django.db import connection
counter = Counter()

now = pd.Timestamp.utcnow()
_from = (now - pd.DateOffset(days=1)).date()
news = Newsdata.objects.filter(published_at__date__gte=_from)

n = news.count()
n_done = [0]
status = '\r{pct}% completed'

def progress():
    n_done[0] += 1
    pct = '%.2f' % (n_done[0] / n * 100)
    print(status.format(pct=pct), end='')

s = time.time()
[counter.update(item) for item in news.values_list('words', flat=True).iterator()]
    
print('\n', time.time() - s)


 7.707777500152588


In [12]:
counter.most_common(100)

[('china', 761),
 ('wednesday', 748),
 ('people', 617),
 ('thursday', 548),
 ('t', 463),
 ('the united states', 401),
 ('the company', 302),
 ('parliament', 286),
 ('july', 262),
 ('the world', 261),
 ('hong kong', 257),
 ('the government', 256),
 ('time', 256),
 ('a lot', 248),
 ('beijing', 246),
 ('the end', 245),
 ('reuters', 233),
 ('investors', 226),
 ('washington', 224),
 ('britain', 222),
 ('brexit', 220),
 ('the country', 207),
 ('the bill', 206),
 ('august', 200),
 ('the time', 194),
 ('the city', 190),
 ('the us', 190),
 ('place', 183),
 ('a no deal brexit', 180),
 ('trump', 180),
 ('a statement', 180),
 ('talks', 179),
 ('oct', 178),
 ('things', 174),
 ('the dollar', 173),
 ('johnson', 171),
 ('june', 164),
 ('women', 158),
 ('asia', 153),
 ('a deal', 150),
 ('october', 148),
 ('cent', 147),
 ('u', 146),
 ('the year', 144),
 ('tuesday', 141),
 ('australia', 139),
 ('day', 138),
 ('prices', 137),
 ('the european union', 135),
 ('aug', 135),
 ('an election', 134),
 ('biden', 1

In [85]:
from django.db.models import F, Func, Count, Max, Sum, Min
list(news.values('pub').annotate(cnt=Count('pub')))

[{'pub': 'time', 'cnt': 7},
 {'pub': 'businessinsider', 'cnt': 177},
 {'pub': 'townhall', 'cnt': 16},
 {'pub': 'npr', 'cnt': 7},
 {'pub': 'cbc', 'cnt': 19},
 {'pub': 'realclearpolitics', 'cnt': 1},
 {'pub': 'heritage', 'cnt': 9},
 {'pub': 'foxnews', 'cnt': 45},
 {'pub': 'huffpost', 'cnt': 12},
 {'pub': 'politico', 'cnt': 9},
 {'pub': 'yahoo', 'cnt': 33},
 {'pub': 'washingtontimes', 'cnt': 49},
 {'pub': 'wsj', 'cnt': 24},
 {'pub': 'nyt', 'cnt': 12},
 {'pub': 'dailybeast', 'cnt': 15},
 {'pub': 'cnbc', 'cnt': 20},
 {'pub': 'reuters', 'cnt': 422},
 {'pub': 'thestreet', 'cnt': 2},
 {'pub': 'zdnet', 'cnt': 18},
 {'pub': 'foxbusiness', 'cnt': 4},
 {'pub': 'newrepublic', 'cnt': 6},
 {'pub': 'washingtonpost', 'cnt': 28},
 {'pub': 'nymag', 'cnt': 7},
 {'pub': 'abcnews', 'cnt': 10},
 {'pub': 'bbc', 'cnt': 46},
 {'pub': 'vice', 'cnt': 15},
 {'pub': 'morningstar', 'cnt': 18},
 {'pub': 'marketwatch', 'cnt': 22},
 {'pub': 'atlantic', 'cnt': 18},
 {'pub': 'guardian', 'cnt': 67},
 {'pub': 'independent'

In [3]:
import asyncio
from functools import partial

def collect(pubs):
    s = time.time()
    n = len(pubs)
    n_done = [0]
    status = '\r{pct}% completed'
    cnt = Counter()
    qry = lambda pub: sum(news.filter(pub__in=pub).values_list('words', flat=True), [])
    
    def progress():
        n_done[0] += 1  # 그냥 n_done으로는 외부에서 변수값을 바꿀수 없으므로
        pct = '%.2f' % (n_done[0] / n * 100)
        print(status.format(pct=pct), end='')
        
    
    async def get(pub):
        resp = await loop.run_in_executor(None, qry, pub)
        #[cnt.update(words) for words in resp]
        #cnt.update(sum(resp, []))
        cnt.update(resp)
        progress()


    async def main():
        fts = [asyncio.ensure_future(get(pub)) for pub in pubs]
        await asyncio.gather(*fts)


    asyncio.set_event_loop(asyncio.new_event_loop())
    loop = asyncio.get_event_loop()

    try:
        # 다음 코드를 주피터에서 돌리려면, tornado를 downgrade 해야함
        # pip install tornado==4.5.3
        loop.run_until_complete(main())

    except Exception as ex:
        print(ex)

    finally:
        loop.close()

    print('\nDONE: {0:.2f} seconds'.format(time.time() - s))
    return cnt

In [8]:
pubs = list(news.values_list('pub', flat=True).distinct())

In [18]:
pubs2 = [pubs[i:i+1] for i in range(len(pubs))[::1]]; pubs2

[['time'],
 ['businessinsider'],
 ['townhall'],
 ['npr'],
 ['cbc'],
 ['realclearpolitics'],
 ['heritage'],
 ['foxnews'],
 ['huffpost'],
 ['politico'],
 ['yahoo'],
 ['washingtontimes'],
 ['wsj'],
 ['nyt'],
 ['dailybeast'],
 ['cnbc'],
 ['reuters'],
 ['thestreet'],
 ['zdnet'],
 ['foxbusiness'],
 ['newrepublic'],
 ['washingtonpost'],
 ['nymag'],
 ['abcnews'],
 ['bbc'],
 ['vice'],
 ['morningstar'],
 ['marketwatch'],
 ['atlantic'],
 ['guardian'],
 ['independent'],
 ['cbs'],
 ['investingcom'],
 ['cnn']]

In [ ]:
kk = collect(pubs2)

97.06% completed